In [1]:
import glob
from PIL import Image, ImageOps
from src.create_annotations import *
import cv2

# Label ids of the dataset
category_ids = {
    # "outlier": 0,
    "rebar": 1,
    "spall": 2,
    "crack": 3,
}

# Define which colors match which categories in the images
category_colors = {
    # "(0, 0, 0)": 0, # Outlier
    "(255, 0, 0)": 1, # "rebar"
    "(0, 255, 0)": 2, # "spall"
    "(0, 0, 255)": 3, # "crack"
}

# Define the ids that are a multiplolygon. In our case: wall, roof and sky
# multipolygon_ids = [ 0, 1, 2]
multipolygon_ids = [5]

def add_margin(pil_img, top, right, bottom, left, color):
    width, height = pil_img.size
    new_width = width + right + left
    new_height = height + top + bottom
    result = Image.new(pil_img.mode, (new_width, new_height), color)
    result.paste(pil_img, (left, top))
    return result

# Get "images" and "annotations" info 
def images_annotations_info(root_path):
    # This id will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    annotations = []
    images = []

    for main_image in glob.glob(root_path + "images/"+ "*.jpg"):
        # print(main_image)
        main_img = Image.open(main_image)
        width, height = main_img.size
        # print("main_img.size", main_img.size)
        # The mask image is *.png but the original image is *.jpg.
        # We make a reference to the original file in the COCO JSON file
        filename = os.path.basename(main_image).split(".")[0]
        rebarmask_file = os.path.join(root_path, "masks",filename + 'rebar' + '.jpg')
        spallmask_file = os.path.join(root_path, "masks", filename + 'spall' + '.jpg')
        crackmask_file = os.path.join(root_path, "masks", filename + 'crack' + '.jpg')
        rebarmask = np.zeros_like(main_img)
        spallmask = np.zeros_like(main_img)
        crackmask = np.zeros_like(main_img)
        # Open the image and (to be sure) we convert it to RGB
        thresh = 150
        fn = lambda x : 255 if x > thresh else 0
        if os.path.exists(rebarmask_file):
            mask1 = cv2.imread(rebarmask_file)
            Conv_hsv_Gray = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
            ret,thresh1 = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
            rebarmask[thresh1 == 255] = [255,0,0]
        if os.path.exists(spallmask_file):
            mask2 = cv2.imread(spallmask_file)#.convert("1")
            Conv_hsv_Gray = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)
            ret,thresh2 = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
            spallmask[thresh2 == 255] = [0,255,0]
            # spallmask = ImageOps.colorize(Image.fromarray(thresh2).convert("RGB"), black =(0,0,0), white =(255, 0, 0))
            # spallmask = add_margin(ImageOps.colorize(mask2, black =(0,0,0), white =(0, 255, 0)),1,1,1,1,(0,0,0))#.point(fn, mode='RGB')
        if os.path.exists(crackmask_file):
            # mask3 = Image.open(crackmask_file)#.convert("1")
            mask3 = cv2.imread(crackmask_file)#.convert("1")
            Conv_hsv_Gray = cv2.cvtColor(mask3, cv2.COLOR_BGR2GRAY)
            ret,thresh3 = cv2.threshold(Conv_hsv_Gray,127,255,cv2.THRESH_BINARY)
            crackmask[thresh3 == 255] = [0,0,255]
            # crackmask = ImageOps.colorize(Image.fromarray(thresh3).convert("RGB"), black =(0,0,0), white =(255, 0, 0))
            # crackmask = add_margin(ImageOps.colorize(mask3, black =(0,0,0), white =(0, 0, 255)),1,1,1,1,(0,0,0))#.point(fn, mode='RGB')
        # mask_image_open = Image.merge("RGB",(rebarmask,spallmask,crackmask))
        # mask_image_open = Image.merge(tmp,crackmask)
        # print(f'rebarmask {type(rebarmask)} spallmask {type(spallmask)} crackmask {type(crackmask)}')
        # pix = np.array(rebarmask)+np.array(spallmask)+np.array(crackmask)
        try:
            pix = rebarmask + spallmask + crackmask
        except:
            print(main_image)
            assert "Error in shape of images loaded"
        mask_image_open = Image.fromarray(pix)
        # mask_image_open = add_margin(mask_image_open_org,1,1,1,1,(0,0,0))#.point(fn, mode='RGB')
        # print("mask_image_open", type(mask_image_open))
        # print(np.unique(pix, return_counts=True))
        # print(np.unique(pix, return_counts=True))
        # break
        # im_new = add_margin(im, 50, 10, 0, 100, (128, 0, 64))

        # print(np.unique(pix, axis=0))
        # print(pix.shape())
        # print(mask_image_open.histogram())
        # mask_image_open.show()
        # break
        # "images" info
        image = create_image_annotation(filename, width, height, image_id)
        images.append(image)

        sub_masks = create_sub_masks(mask_image_open, width, height)
        # print(sub_masks)
        # break
        # sub_masks = {'(255, 0, 0)':rebarmask.getdata(), '(0, 255, 0)': spallmask.getdata(), '(0, 0, 255)': crackmask.getdata()}
        for color, sub_mask in sub_masks.items():
            if color in category_colors.keys():
                category_id = category_colors[color]
            elif color == "(0, 0, 0)":
                continue
            else:
                print("color", color)
                print("root_path", main_image)
                res = eval(color)
                category_id = res.index(255)
            # "annotations" info
            polygons, segmentations = create_sub_mask_annotation(sub_mask)
            # print(segmentations)
            # Check if we have classes that are a multipolygon
            if category_id in multipolygon_ids:
                # Combine the polygons to calculate the bounding box and area
                multi_poly = MultiPolygon(polygons)
                print("multi_poly.bounds",multi_poly.bounds)
                annotation = create_annotation_format(multi_poly, segmentations, image_id, category_id, annotation_id)

                annotations.append(annotation)
                annotation_id += 1
            else:
                # print(len(polygons))
                for i in range(len(polygons)):
                    # Cleaner to recalculate this variable
                    if polygons[i].geom_type == 'Polygon':
                      segmentation = [np.array(polygons[i].exterior.coords).ravel().tolist()]
                    if polygons[i].geom_type == 'MultiPolygon':
                      segmentation = [list(x.exterior.coords) for x in polygons[i].geoms]
                    
                    annotation = create_annotation_format(polygons[i], segmentation, image_id, category_id, annotation_id)
                    
                    annotations.append(annotation)
                    annotation_id += 1
        image_id += 1
    return images, annotations, annotation_id


In [2]:
if __name__ == "__main__":
    # Get the standard COCO JSON format
    coco_format = get_coco_json_format()
    root = "../../data/"
    # root = "../../../Dataset/buildings/Building dataset/"
    
    for keyword in ["train"]:
        root_path = root+"dataset/"
        mask_path = root_path + "masks"
        
        # Create category section
        coco_format["categories"] = create_category_annotation(category_ids)
    
        # Create images and annotations sections
        coco_format["images"], coco_format["annotations"], annotation_cnt = images_annotations_info(root_path)

        with open("output/{}.json".format(keyword),"w") as outfile:
            json.dump(coco_format, outfile)
        
        print("Created %d annotations for images in folder: %s" % (annotation_cnt, mask_path))

color (255, 0, 255)
root_path ../../data/dataset/images/33-5.jpg
Created 2212 annotations for images in folder: ../../data/dataset/masks
